In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [ ]:
class FinancialStatements: 
    
    def getelementinlist(list,element):
        try:
            return list[element]
        except:
            return '-'
    
    def getFinancialStatements(self,Stocks, dates, root, folder, form):
        i = 0
        no_of_stocks = len(Stocks.index)
        for i in range(no_of_stocks):
            try:
                urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials'
                urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials/balance-sheet'
                urlcashflow = 'https://www.marketwatch.com/investing/stock/'+ Stocks.iloc[i]['Symbols'] +'/financials/cash-flow'
                text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"lxml")
                text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"lxml")
                text_soup_cashflow = BeautifulSoup(requests.get(urlcashflow).text,"lxml")

                # Income Statement
                titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
                saleslist = []
                grossincomelist = []
                pretaxincomelist = []
                netincomelist = []
                ebitdalist= []
                for title in titlesfinancials:
                    if 'Sales/Revenue' in title.text:
                        saleslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Gross Income' in title.text:
                        grossincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Pretax Income' in title.text:
                        pretaxincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Income' in title.text:
                        netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'EBITDA' in title.text:
                        ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Balance sheet
                titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
                cashshortlist = []
                totalassetslist = []
                shorttermdebtlist = []
                totalliabilitieslist = [] 
                totalequitylist = []
                for title in titlesbalancesheet:
                    if 'Cash & Short Term Investments' in title.text:
                        cashshortlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Assets' in title.text:
                        totalassetslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Short Term Debt' in title.text:
                        shorttermdebtlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Liabilities' in title.text:
                        totalliabilitieslist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Total Equity' in title.text:
                        totalequitylist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Cash Flow
                titlescashflow = text_soup_cashflow.findAll('td', {'class': 'rowTitle'})
                netoperatingcashflowlist = []
                netinvestingcashflowlist = []
                netfinincingcashflowlist = []
                freecashflowlist = []
                for title in titlescashflow:
                    if 'Net Operating Cash Flow' in title.text:
                        netoperatingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Investing Cash Flow' in title.text:
                        netinvestingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Net Financing Cash Flow' in title.text:
                        netfinincingcashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
                    if 'Free Cash Flow' in title.text:
                        freecashflowlist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])

                # Income Statement
                sales = getelementinlist(saleslist,0)
                grossincome = getelementinlist(grossincomelist,0)
                pretaxincome = getelementinlist(pretaxincomelist,0)
                netincome = getelementinlist(netincomelist,0)
                ebitda = getelementinlist(ebitdalist,0)

                #Balance Sheet
                cashshort = getelementinlist(cashshortlist,0)
                totalassets = getelementinlist(totalassetslist,1)
                shorttermdebt = getelementinlist(shorttermdebtlist,0)
                totalliabilities = getelementinlist(totalliabilitieslist,0)
                totalequity = getelementinlist(totalequitylist,0)

                #Cash Flow
                netoperatingcashflow = getelementinlist(netoperatingcashflowlist,0)
                netinvestingcashflow = getelementinlist(netinvestingcashflowlist,0)
                netfinancingcashflow = getelementinlist(netfinincingcashflowlist,0)
                freecashflow = getelementinlist(freecashflowlist,0)

                self.financial_statements= pd.DataFrame({'sales': sales,'grossincome': grossincome,'pretaxincome': pretaxincome,
                                  'netincome': netincome,'ebitda': ebitda, 'cashshort': cashshort, 
                                  'totalassets': totalassets,'shorttermdebt': shorttermdebt, 'totalliabilities': totalliabilities,
                                  'totalequity': totalequity, 'netoperatingcashflow': netoperatingcashflow, 
                                  'netinvestingcashflow': netinvestingcashflow, 'netfinancingcashflow': netfinancingcashflow,  
                                  'freecashflow': freecashflow}, index = dates)
                self.financial_statements.to_csv(root + folder + Stocks.iloc[i]['Symbols']+ 'FinancialStatements' + form, index = True, header=True)
            except:
                print('An error occurred on ' + Stocks.iloc[i]['Symbols'])

In [ ]:
Stocks = pd.read_csv('./Stocks.csv')
dates = [2015,2016,2017,2018,2019]
root = './Stocks/'
folder = ''
form = '.csv'

In [ ]:
financialStatements = FinancialStatements()
financialStatements.getFinancialStatements(Stocks, dates, root, folder, form)
print(financialStatements.financial_statements.iloc[3])
a = float(re.sub(r"B","",financialStatements.financial_statements['freecashflow'][2017]))
print(a * 1000000000)